# Lab 4: Task Dependencies và Branching - Quản lý Workflow Logic

## 🎯 Objectives
- Hiểu cách định nghĩa task dependencies với bitshift operators
- Sử dụng BranchPythonOperator cho conditional logic
- Áp dụng trigger rules để xử lý failures
- Tạo dynamic tasks với task mapping
- Sử dụng TaskGroups để tổ chức tasks
- Xây dựng conditional workflows phức tạp

## 📋 Prerequisites
- Hoàn thành Lab 1-3
- Hiểu operators và tasks
- Airflow cluster đang chạy

## 🏗️ Dependencies Overview
Task dependencies trong Airflow có thể được định nghĩa bằng:
- **Bitshift operators**: `>>` (set downstream), `<<` (set upstream)
- **Methods**: `set_downstream()`, `set_upstream()`
- **Lists**: Multiple dependencies cùng lúc


## 1. Import Libraries và Setup


In [ ]:
# Import Airflow dependencies và branching
from airflow.sdk import DAG, task
from airflow.providers.standard.operators.empty import EmptyOperator
from airflow.providers.standard.operators.python import BranchPythonOperator, PythonOperator
from airflow.providers.standard.operators.bash import BashOperator
from airflow.sdk.task_group import TaskGroup

import pendulum
from datetime import datetime, timedelta
import random

print("✅ Airflow dependencies và branching modules imported successfully!")


## 2. Bitshift Operators - Định nghĩa Dependencies

Airflow sử dụng bitshift operators (`>>` và `<<`) để định nghĩa dependencies một cách trực quan và Pythonic.


In [ ]:
# DAG với bitshift operators
@dag(
    dag_id="bitshift_dependencies_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["dependencies", "bitshift"],
)
def bitshift_dependencies_dag():
    """
    ### Bitshift Operators Example
    DAG mẫu sử dụng bitshift operators để định nghĩa dependencies.
    """
    
    # Tạo tasks
    start = EmptyOperator(task_id="start")
    task_a = EmptyOperator(task_id="task_a")
    task_b = EmptyOperator(task_id="task_b")
    task_c = EmptyOperator(task_id="task_c")
    task_d = EmptyOperator(task_id="task_d")
    end = EmptyOperator(task_id="end")
    
    # Cách 1: Single dependency với >>
    # start >> task_a có nghĩa là task_a phụ thuộc vào start
    start >> task_a
    
    # Cách 2: Multiple downstream tasks
    # task_a chạy trước task_b và task_c
    task_a >> [task_b, task_c]
    
    # Cách 3: Multiple upstream tasks
    # task_d phụ thuộc vào cả task_b và task_c
    [task_b, task_c] >> task_d
    
    # Cách 4: Chain dependencies
    task_d >> end
    
    # Hoặc viết gọn hơn:
    # start >> task_a >> [task_b, task_c] >> task_d >> end

# Create DAG
bitshift_dag = bitshift_dependencies_dag()

print("✅ Bitshift Dependencies DAG created!")
print(f"Tasks: {[task.task_id for task in bitshift_dag.tasks]}")
print("\n📊 Dependency Flow:")
print("start → task_a → [task_b, task_c] → task_d → end")


## 3. BranchPythonOperator - Conditional Branching

BranchPythonOperator cho phép chọn nhánh nào sẽ chạy dựa trên điều kiện. Function phải return task_id(s) của task(s) tiếp theo.


In [ ]:
# DAG với BranchPythonOperator
@dag(
    dag_id="branching_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["branching", "conditional"],
)
def branching_dag():
    """
    ### Branching Example
    DAG mẫu sử dụng BranchPythonOperator cho conditional logic.
    """
    
    start = EmptyOperator(task_id="start")
    
    # Branch function - return task_id(s) của task(s) sẽ chạy
    def choose_branch(**context):
        """Chọn branch dựa trên điều kiện"""
        execution_date = context['data_interval_start']
        day_of_week = execution_date.weekday()
        
        # Thứ 2 (0) hoặc thứ 6 (4): chạy cả 2 branches
        if day_of_week == 0 or day_of_week == 4:
            return ["branch_a", "branch_b"]
        # Cuối tuần: chỉ chạy branch_a
        elif day_of_week >= 5:
            return "branch_a"
        # Ngày thường: chỉ chạy branch_b
        else:
            return "branch_b"
    
    branching = BranchPythonOperator(
        task_id="branching",
        python_callable=choose_branch,
    )
    
    # Các branch tasks
    branch_a = EmptyOperator(task_id="branch_a")
    branch_b = EmptyOperator(task_id="branch_b")
    
    # Join task - chạy sau khi các branches hoàn thành
    join = EmptyOperator(
        task_id="join",
        trigger_rule="none_failed_min_one_success",  # Chạy nếu ít nhất 1 branch thành công
    )
    
    end = EmptyOperator(task_id="end")
    
    # Define dependencies
    start >> branching
    branching >> branch_a >> join
    branching >> branch_b >> join
    join >> end

# Create DAG
branching_dag_instance = branching_dag()

print("✅ Branching DAG created!")
print(f"Tasks: {[task.task_id for task in branching_dag_instance.tasks]}")
print("\n💡 Branch logic:")
print("  - Monday/Friday: Run both branches")
print("  - Weekend: Run branch_a only")
print("  - Weekday: Run branch_b only")


## 4. Trigger Rules - Xử lý Task Failures

Trigger rules xác định khi nào một task sẽ chạy dựa trên trạng thái của upstream tasks. Các trigger rules phổ biến:
- `all_success` (default): Tất cả upstream tasks phải thành công
- `all_failed`: Tất cả upstream tasks phải failed
- `all_done`: Chạy khi tất cả upstream tasks hoàn thành (bất kể status)
- `one_failed`: Chạy khi ít nhất 1 upstream task failed
- `one_success`: Chạy khi ít nhất 1 upstream task thành công
- `none_failed`: Chạy khi không có upstream task nào failed (success hoặc skipped)
- `none_failed_min_one_success`: Chạy khi không có failed và ít nhất 1 success
- `none_skipped`: Chạy khi không có upstream task nào skipped


In [ ]:
# DAG với Trigger Rules
@dag(
    dag_id="trigger_rules_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["trigger-rules", "error-handling"],
)
def trigger_rules_dag():
    """
    ### Trigger Rules Example
    DAG mẫu minh họa các trigger rules khác nhau.
    """
    
    start = EmptyOperator(task_id="start")
    
    # Task có thể fail
    def task_that_might_fail(**context):
        """Task có thể fail dựa trên điều kiện"""
        import random
        if random.random() < 0.5:
            raise Exception("Random failure occurred")
        return "Success"
    
    might_fail = PythonOperator(
        task_id="might_fail",
        python_callable=task_that_might_fail,
    )
    
    # Task luôn thành công
    always_succeed = EmptyOperator(task_id="always_succeed")
    
    # Task với trigger_rule="all_done" - chạy bất kể upstream status
    cleanup_all_done = EmptyOperator(
        task_id="cleanup_all_done",
        trigger_rule="all_done",
    )
    
    # Task với trigger_rule="one_failed" - chạy nếu có task failed
    handle_failure = EmptyOperator(
        task_id="handle_failure",
        trigger_rule="one_failed",
    )
    
    # Task với trigger_rule="none_failed_min_one_success" - chạy nếu không failed và có success
    final_task = EmptyOperator(
        task_id="final_task",
        trigger_rule="none_failed_min_one_success",
    )
    
    end = EmptyOperator(
        task_id="end",
        trigger_rule="all_done",  # Luôn chạy
    )
    
    # Define dependencies
    start >> [might_fail, always_succeed]
    [might_fail, always_succeed] >> cleanup_all_done
    might_fail >> handle_failure
    [cleanup_all_done, handle_failure] >> final_task >> end

# Create DAG
trigger_rules_dag_instance = trigger_rules_dag()

print("✅ Trigger Rules DAG created!")
print(f"Tasks: {[task.task_id for task in trigger_rules_dag_instance.tasks]}")
print("\n📊 Trigger Rules:")
print("  - cleanup_all_done: all_done (chạy bất kể status)")
print("  - handle_failure: one_failed (chạy nếu có failure)")
print("  - final_task: none_failed_min_one_success (chạy nếu không failed và có success)")
print("  - end: all_done (luôn chạy)")


## 5. Dynamic Task Mapping - Tạo Tasks Động

Dynamic Task Mapping cho phép tạo nhiều task instances từ một task definition dựa trên input data. Rất hữu ích cho:
- Processing multiple files
- Batch operations
- Parallel processing


In [ ]:
# DAG với Dynamic Task Mapping
@dag(
    dag_id="dynamic_task_mapping_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["dynamic", "mapping"],
)
def dynamic_mapping_dag():
    """
    ### Dynamic Task Mapping Example
    DAG mẫu sử dụng dynamic task mapping để tạo tasks động.
    """
    
    # Task tạo list các items cần process
    @task
    def get_items_to_process():
        """Trả về list các items cần process"""
        items = [
            {"id": 1, "name": "item_1", "value": 100},
            {"id": 2, "name": "item_2", "value": 200},
            {"id": 3, "name": "item_3", "value": 300},
        ]
        print(f"Generated {len(items)} items to process")
        return items
    
    # Task với dynamic mapping - sẽ tạo nhiều task instances
    @task
    def process_item(item: dict):
        """Process một item - sẽ được map cho mỗi item"""
        print(f"Processing {item['name']} with value {item['value']}")
        result = item['value'] * 2
        print(f"Result: {result}")
        return {"item_id": item['id'], "result": result}
    
    # Task tổng hợp kết quả
    @task
    def aggregate_results(results: list):
        """Tổng hợp kết quả từ tất cả tasks"""
        total = sum(r['result'] for r in results)
        print(f"Aggregated total: {total}")
        return total
    
    # Define workflow với dynamic mapping
    items = get_items_to_process()
    # .expand() tạo dynamic task instances cho mỗi item
    processed_items = process_item.expand(item=items)
    aggregate_results(processed_items)

# Create DAG
dynamic_mapping_dag_instance = dynamic_mapping_dag()

print("✅ Dynamic Task Mapping DAG created!")
print("\n💡 Dynamic mapping sẽ tạo:")
print("  - 1 task instance cho get_items_to_process")
print("  - 3 task instances cho process_item (một cho mỗi item)")
print("  - 1 task instance cho aggregate_results")


## 6. TaskGroups - Tổ chức Tasks

TaskGroups cho phép nhóm các tasks lại với nhau để:
- Cải thiện visualization trong UI
- Tổ chức code tốt hơn
- Áp dụng settings cho nhóm tasks
- Tạo sub-workflows


In [ ]:
# DAG với TaskGroups
@dag(
    dag_id="taskgroup_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["taskgroup", "organization"],
)
def taskgroup_dag():
    """
    ### TaskGroup Example
    DAG mẫu sử dụng TaskGroups để tổ chức tasks.
    """
    
    start = EmptyOperator(task_id="start")
    
    # TaskGroup 1: Data Extraction
    with TaskGroup("extract_group") as extract_group:
        extract_a = EmptyOperator(task_id="extract_source_a")
        extract_b = EmptyOperator(task_id="extract_source_b")
        extract_c = EmptyOperator(task_id="extract_source_c")
        
        # Dependencies trong group
        extract_a >> [extract_b, extract_c]
    
    # TaskGroup 2: Data Transformation
    with TaskGroup("transform_group") as transform_group:
        transform_a = EmptyOperator(task_id="transform_a")
        transform_b = EmptyOperator(task_id="transform_b")
        
        transform_a >> transform_b
    
    # TaskGroup 3: Data Loading
    with TaskGroup("load_group") as load_group:
        load_a = EmptyOperator(task_id="load_destination_a")
        load_b = EmptyOperator(task_id="load_destination_b")
        
        # Parallel loading
        [load_a, load_b]
    
    end = EmptyOperator(task_id="end")
    
    # Define dependencies giữa groups
    start >> extract_group >> transform_group >> load_group >> end

# Create DAG
taskgroup_dag_instance = taskgroup_dag()

print("✅ TaskGroup DAG created!")
print(f"Tasks: {[task.task_id for task in taskgroup_dag_instance.tasks]}")
print("\n📊 Task Groups:")
print("  - extract_group: extract_source_a → [extract_source_b, extract_source_c]")
print("  - transform_group: transform_a → transform_b")
print("  - load_group: [load_destination_a, load_destination_b] (parallel)")


## 7. Complex Branching Scenario - Real-world Example

Tạo một workflow phức tạp với nhiều branches và conditions để minh họa cách sử dụng branching trong thực tế.


In [ ]:
# Complex Branching Scenario
@dag(
    dag_id="complex_branching_scenario",
    schedule="@daily",
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["complex", "branching", "real-world"],
)
def complex_branching_dag():
    """
    ### Complex Branching Scenario
    DAG mẫu với branching logic phức tạp cho use case thực tế.
    """
    
    start = EmptyOperator(task_id="start")
    
    # Check data quality
    def check_data_quality(**context):
        """Kiểm tra chất lượng data"""
        # Simulate data quality check
        import random
        quality_score = random.uniform(0.7, 1.0)
        
        if quality_score >= 0.9:
            return "high_quality"
        elif quality_score >= 0.8:
            return "medium_quality"
        else:
            return "low_quality"
    
    quality_check = BranchPythonOperator(
        task_id="check_data_quality",
        python_callable=check_data_quality,
    )
    
    # Different processing paths based on quality
    high_quality_process = EmptyOperator(task_id="high_quality_process")
    medium_quality_process = EmptyOperator(task_id="medium_quality_process")
    low_quality_process = EmptyOperator(task_id="low_quality_process")
    
    # Data cleaning for low quality
    clean_data = EmptyOperator(task_id="clean_data")
    low_quality_process >> clean_data
    
    # Join after processing
    join_processing = EmptyOperator(
        task_id="join_processing",
        trigger_rule="none_failed_min_one_success",
    )
    
    # Final validation
    validate_output = EmptyOperator(task_id="validate_output")
    
    # Notification based on result
    def send_notification(**context):
        """Gửi notification dựa trên kết quả"""
        ti = context['ti']
        # Check if validation passed
        print("Sending notification...")
        return "notification_sent"
    
    send_notif = PythonOperator(
        task_id="send_notification",
        python_callable=send_notification,
    )
    
    end = EmptyOperator(
        task_id="end",
        trigger_rule="all_done",
    )
    
    # Define dependencies
    start >> quality_check
    quality_check >> high_quality_process >> join_processing
    quality_check >> medium_quality_process >> join_processing
    quality_check >> low_quality_process
    clean_data >> join_processing
    join_processing >> validate_output >> send_notif >> end

# Create DAG
complex_branching_dag_instance = complex_branching_dag()

print("✅ Complex Branching DAG created!")
print(f"Tasks: {[task.task_id for task in complex_branching_dag_instance.tasks]}")
print("\n📊 Workflow:")
print("  start → check_data_quality → [high/medium/low_quality_process]")
print("  low_quality_process → clean_data → join_processing")
print("  join_processing → validate_output → send_notification → end")


## 9. Tóm tắt và Next Steps

### ✅ Những gì đã học:
1. Bitshift operators (>>, <<) - Định nghĩa dependencies
2. BranchPythonOperator - Conditional branching
3. Trigger rules - Xử lý failures và skipped tasks
4. Dynamic Task Mapping - Tạo tasks động
5. TaskGroups - Tổ chức tasks
6. Complex branching scenarios - Use cases thực tế
7. Best practices và common pitfalls

### 📚 Next Lab:
- **Lab 5**: XCom và Data Sharing
- XCom push/pull
- Task return values
- Custom XCom backends
- Data passing best practices

### 🔗 Useful Links:
- [Task Dependencies](https://airflow.apache.org/docs/apache-airflow/3.1.1/core-concepts/dags.html#task-dependencies)
- [Branching](https://airflow.apache.org/docs/apache-airflow/3.1.1/core-concepts/dags.html#branching)
- [Trigger Rules](https://airflow.apache.org/docs/apache-airflow/3.1.1/core-concepts/tasks.html#trigger-rules)
- [Dynamic Task Mapping](https://airflow.apache.org/docs/apache-airflow/3.1.1/core-concepts/dynamic-task-mapping.html)
- [TaskGroups](https://airflow.apache.org/docs/apache-airflow/3.1.1/core-concepts/taskgroup.html)

### 💡 Exercises:
1. Tạo DAG với branching dựa trên data quality
2. Sử dụng dynamic mapping để process multiple files
3. Tạo TaskGroups cho ETL pipeline
4. Implement error handling với trigger rules
5. Xây dựng complex workflow với nhiều branches
